### Law Question Pair Notebook<br>
**Project3-20231**<br>
<font size="3">**Student:** Nguyen Van-Tho 20204694<br></font>
<font size="3">**Lecturer:** Ph. D. Tran Van Dang, Department of Computer Science, SoICT
</font>

<font size="2.5">**Mô tả đề tài:** Sử dụng unsupervised-SimCSE để fine-tuning BERT, cho phép tìm kiếm câu hỏi tương đồng mức ngữ nghĩa.<br></font>
<font size="2.5">**Dữ liệu:** Thu thập từ website https://thuvienphapluat.vn/hoi-dap-phap-luat/ bằng công cụ Scrapy.</font>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Import thư viện
<font size="2.5">Cài đặt thư viện cần thiết để chạy mô hình: sentence-transformers, dataset.</font>


In [ ]:
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 14.4 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=0c22800fa7edf01766617c6ec005370e38fc620fff94614f4f0f603c79dcfb52
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [ ]:
!pip -q install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.1 MB/s eta 0:00:00


In [ ]:
import json
from torch.utils.data import DataLoader
from datasets import Dataset
import math
from sentence_transformers import models, losses, util
from sentence_transformers import LoggingHandler, SentenceTransformer, InputExample
from datetime import datetime
import gzip
import sys
import tqdm

#### Huấn luyện BERT với SimCSE-không giám sát (unsupervised-SimCSE)<br>
<font size="2">**Dữ liệu đầu vào:** Tập các câu hỏi của người dùng<br></font>
<font size="2">**Mô hình tinh chỉnh:** *multilingual-e5-base* like BERT<br></font>

In [ ]:
file_dir = "/content/drive/MyDrive/pair-question.jsonl"

In [ ]:
# Training parameters
model_name = "intfloat/multilingual-e5-base"
train_batch_size = 8
num_epochs = 1

In [ ]:
model_output_path = ("/content/"+model_name.replace("/", "-") + datetime.now().strftime("%Y-%m-%d_%H-%M-%S"))

word_embedding_model = models.Transformer(model_name)
pooling_model = models.Pooling(
    word_embedding_model.get_word_embedding_dimension(),
    pooling_mode_mean_tokens=True,
    pooling_mode_cls_token=False,
    pooling_mode_max_tokens=False,
)
model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

In [ ]:
print(model_output_path)

/content/intfloat-multilingual-e5-base2024-01-12_23-10-58


In [ ]:
# Read the train corpus
train_samples = []
origin_ds = Dataset.from_json(file_dir).train_test_split(test_size=0.1, seed=42)
for row in origin_ds["train"]:
    train_samples.append(InputExample(texts=[row["user_question"], row["user_question"]]))

# Train model using the MultipleNegativesRankingLoss
train_dataloader = DataLoader(train_samples, shuffle=True, batch_size=train_batch_size, drop_last=True)
train_loss = losses.MultipleNegativesRankingLoss(model)

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
# Train the model
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=num_epochs,
    optimizer_params={"lr": 5e-5},
    checkpoint_path=model_output_path,
    show_progress_bar=True,
    use_amp=False,  # Set to True, if your GPU supports FP16 cores
)

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/14734 [00:00<?, ?it/s]

In [ ]:
!zip -r /content/drive/MyDrive/model.zip /content/intfloat-multilingual-e5-base2024-01-12_23-10-58/14734

  adding: content/intfloat-multilingual-e5-base2024-01-12_23-10-58/14734/ (stored 0%)
  adding: content/intfloat-multilingual-e5-base2024-01-12_23-10-58/14734/model.safetensors (deflated 25%)
  adding: content/intfloat-multilingual-e5-base2024-01-12_23-10-58/14734/1_Pooling/ (stored 0%)
  adding: content/intfloat-multilingual-e5-base2024-01-12_23-10-58/14734/1_Pooling/config.json (deflated 47%)
  adding: content/intfloat-multilingual-e5-base2024-01-12_23-10-58/14734/special_tokens_map.json (deflated 52%)
  adding: content/intfloat-multilingual-e5-base2024-01-12_23-10-58/14734/sentencepiece.bpe.model (deflated 49%)
  adding: content/intfloat-multilingual-e5-base2024-01-12_23-10-58/14734/config.json (deflated 49%)
  adding: content/intfloat-multilingual-e5-base2024-01-12_23-10-58/14734/README.md (deflated 58%)
  adding: content/intfloat-multilingual-e5-base2024-01-12_23-10-58/14734/config_sentence_transformers.json (deflated 28%)
  adding: content/intfloat-multilingual-e5-base2024-01-12_


### Load model BKAI
<font size="2.5">
Sử dụng pre-trained model của BKAI để lấy nhúng câu (sentences-embedding) phục vụ cho việc tìm các câu có ngữ nghĩa tương đồng.


* <font size="2.5">Về dữ liệu train của model (sau khi đã dịch và tiền xử lý):
 * <font size="2.5">MSMACRO: ~ 4GB
 * <font size="2.5">SQuAD v2: ~ 30 MB
 * <font size="2.5">Legal Text Retrieval Zalo 2021: ~ 200 MB

* <font size="2.5">Thời gian training: ~ 1h / epoch (thông tin được cung cấp bởi BKAI)
</font>

In [ ]:
model = SentenceTransformer('bkai-foundation-models/vietnamese-bi-encoder')

.gitattributes:   0%|          | 0.00/1.70k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/6.08k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/22.0 [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

...

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/303 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

In [ ]:
def read_jsonl(file_path):
    data = []
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            data.append(json.loads(line))
    return data

In [ ]:
data = read_jsonl(file_dir)

In [ ]:
print(data[1])

{'url': 'https://thuvienphapluat.vn/hoi-dap-phap-luat/1F43B-hd-lam-so-do-chung-cu-co-vay-von-ngan-hang.html', 'title': 'Làm sổ đỏ chung cư có vay vốn ngân hàng', 'user_question': 'Xin chào anh/chị, Tôi có mua chung cư (đang xây dựng, chưa bàn giao) có hỗ trợ vay vốn ngân hàng (hợp đồng mua bán với CĐT ngân hàng cầm). Vậy sau này bàn giao nhà với CĐT tôi có thể làm sổ đỏ được không? Nếu tôi không làm sổ đỏ ngay mà sau này tự làm(không qua CĐT) có được không? Xin cám ơn các anh/chị.', 'question': 'Làm sổ đỏ chung cư có vay vốn ngân hàng', 'answer': '- Về phía ngân hàng: Ngân hàng hỗ trợ vốn thì họ sẽ kiểm soát chặt căn hộ vì đó là tài sản bảo đảm cho khoản vay và vì vậy khi nào nghĩa vụ đối với khoản vay được hoàn thành thì mới có thể được cấp giấy chứng nhận liên quan đến căn hộ, trừ khi ngân hàng có thỏa thuận khác với bạn.\n- Về CĐT: Thường trường hợp như bạn nêu, CĐT có nghĩa vụ hoàn thành giấy chứng nhận liên quan đến căn hộ cho bạn. Khi CĐT yêu cầu người mua phối hợp làm thủ tục để

<font size="2">Tiến hành lưu trữ tất cả các câu hỏi có sẵn để phục vụ quá trình tìm kiếm. </font>



In [ ]:
question_data = [item['question'] for item in data]

#### Xử lý dữ liệu
<font size="2">Chuyển các câu hỏi có sẵn thành các embedding<br></font>

In [ ]:
question_data[3]

'Có phải đóng thuế thu nhập cá nhân khi bán quần áo ở chợ?'

In [ ]:
data_embeddings = model.encode(question_data)

In [ ]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(data, test_size=0.1, random_state=42)
print(len(test_data))

13098


In [ ]:
print(test_data[0]['user_question'])

Theo quy định của Luật ngân sách nhà nước thì tôi được biết thì Chính phủ và Uỷ ban nhân dân cấp tỉnh, thành phố trực thuộc trung ương được lập quỹ dự trữ tài chính từ một số nguồn theo quy định của pháp luật. Vậy Ban biên tập cho tôi hỏi: quỹ dự trữ tài chính của Chính phủ và Uỷ ban nhân dân cấp tỉnh được quy định ra sao?


In [ ]:
def get_similarity(question, topk=5):
    question_emb = model.encode(question)
    result = util.semantic_search(question_emb, data_embeddings)
    result = [(x['corpus_id'],x['score']) for x in result[0]]

    result = sorted(result, key=lambda x: x[1], reverse=True)

    topk_result = result[:topk]
    topk_info = [(question_data[i] ,data[i]['answer'], score, data[i]['url'], data[i]['user_question']) for i, score in topk_result]
    return topk_info

In [51]:
question = "Tôi muốn đăng ký tạm trú phải làm thế nào?"
for a,b,c,d,e in get_similarity(question):
    print(f"Question: {a}\n\nAnswer: {b}\n\nScore: {c}\n\nLink: {d}")
    print('-----'*15)

Question: Hướng dẫn thủ tục đăng ký tạm trú

Answer: Ngày 09/9/2014,  Bộ trưởng Bộ Công an ban hành Thông tư 35/2014/TT-BCA hướng dẫn Luật Cư trú và Nghị định 31/2014/NĐ-CP. 
Theo đó, thủ tục đăng ký tạm trú là một trong những nội dung trọng tâm và được quy định tại Điều 16 Thông tư 35/2014/TT-BCA. Cụ thể như sau:



Điều kiện
Người tạm trú được chủ hộ đồng ý cho nhập vào sổ tạm trú hoặc đồng ý cho đăng ký tạm trú tại nơi thường trú của chủ hộ thì việc đồng ý phải được ghi vào phiếu báo thay đổi hộ khẩu, nhân khẩu, ký, ghi rõ họ tên và ngày, tháng, năm.


Hồ sơ

a) Phiếu báo thay đổi hộ khẩu, nhân khẩu; bản khai nhân khẩu (đối với các trường hợp phải khai bản khai nhân khẩu);
b) Giấy tờ, tài liệu chứng minh chỗ ở hợp pháp theo quy định tại Điều 6 Nghị định số 31/2014/NĐ-CP (trừ trường hợp được chủ hộ có sổ hộ khẩu hoặc sổ tạm trú đồng ý cho đăng ký tạm trú thì không cần xuất trình giấy tờ, tài liệu chứng minh chỗ ở hợp pháp). Trường hợp chỗ ở hợp pháp do thuê, mượn, ở nhờ thì phải được

### Đánh giá mô hình
<font size="2"> Sử dụng chỉ số **Precision@K**, **Recall@K**.
* <font size="2"> **Precision@K** đo khả năng của mô hình trả về các câu hỏi chính xác trong top K câu hỏi được đề xuất.</font>
* <font size="2"> **Recall@K** được tính bằng tỉ lệ số lượng câu hỏi đúng trong top K câu hỏi được trả về chia cho tổng số câu hỏi đúng. </font> <br>

Lấy 10% dữ liệu để tiến hành kiểm thử, đo lường hai chỉ số trên. Trong quá trình kiểm thử, sử dụng trường "**user_question**" để đo độ tương đồng với bộ **"question"** định sẵn.
</font>

<font size='2'> Embedding toàn bộ "user_question" của tập test. </font>

In [ ]:
test_x = [item['user_question'] for item in test_data]
test_embeddings = model.encode(test_x)

In [ ]:
result = util.semantic_search(test_embeddings, data_embeddings)

<font size='2'> Đếm tất cả các bộ Q-A có chung user_question, là cơ sở để đo **Recall@K.** </font>

In [ ]:
question_count = {}
def add_question(user_question):
    if user_question in question_count:
        question_count[user_question] += 1
    else:
        question_count[user_question] = 1

def get_count(user_question):
    return question_count.get(user_question, 0)

for item in data:
    add_question(item['user_question'])

In [ ]:
def pre_rec(result, topk):
    total_correct = 0
    total_precision = 0
    total_recall = 0
    for j in range(len(test_data)):
        single = result[j]
        topk_single = single[:topk]
        topk_single_res = [(x['corpus_id'],x['score']) for x in topk_single]
        topk_info = [(question_data[i], data[i]['user_question']) for i, score in topk_single_res]
        topk_user_questions = [z for y, z in topk_info]

        if (topk_user_questions[0] == test_data[j]['user_question']):
            correct = 1
        else:
            correct = 0

        sum_k = sum(1 for q in topk_user_questions[:topk] if q == test_data[j]['user_question'] )
        pre_k = sum_k/ topk
        tmp = get_count(test_data[j]['user_question'])
        if tmp > topk:
          tmp = topk
        rec_k = sum_k/ tmp
        if rec_k > 1:
          rec_k = 1
        total_correct += correct
        total_precision += pre_k
        total_recall += rec_k

    correct_rate = total_correct / len(test_data)
    average_precision = total_precision / len(test_data)
    average_recall = total_recall / len(test_data)
    return [correct_rate, average_precision, average_recall]

In [ ]:
topk = 5
cr, prek, reck = pre_rec(result, topk)

In [ ]:
print(f'Correct rate: {cr}')
print(f'Precision@{topk}: {prek}')
print(f'Recall@{topk}: {reck}')

Correct rate: 0.5671858298976943
Precision@5: 0.1536265078638166
Recall@5: 0.6282600397007166


<font size='2'> **Nhận xét:** <br>
- Với mô hình đã xây dựng và số lượng câu hỏi kết quả đưa ra là 5, chỉ số Recall@5 cao cho thấy trong số 5 kết quả thực hiện thì số lượng các câu hỏi liên quan/có ích đến chủ đề được hỏi xuất hiện thường xuyên hơn. <br>
- Chỉ số Precision@5 thấp không đồng nghĩa với mô hình tệ, do có một số chủ đề (user_question) chỉ có 1 câu hỏi liên quan (format của trang web là cứ một câu hỏi người dùng sẽ có từ 1-4 câu hỏi con có nội dung liên quan), làm cho chỉ số này rất thấp.
</font>

### Thiết kế giao diện demo đơn giản với Gradio
<font size="2"> Về cơ bản, Gradio cung cấp người dùng các công cụ để xây dựng một giao diện demo đơn giản cho các mô hình học máy/học sâu.<br>
**Chi tiết trên HuggingFace:** [Giới thiệu về Gradio](https://huggingface.co/learn/nlp-course/vi/chapter9/1?fw=tf)
</font>

In [ ]:
!pip uninstall fastapi
!pip uninstall gradio
!pip install fastapi==0.100.0
!pip install -q gradio==3.48.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.7/65.7 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 5.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.3/20.3 MB 24.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.2/299.2 kB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
import gradio as gr
import random
import time

with gr.Blocks() as demo:
    chatbot = gr.Chatbot()
    msg = gr.Textbox(placeholder="Nhập tin nhắn vào đây")
    clear = gr.ClearButton([msg, chatbot])
    def generate(question):
      output = ''
      for a,b,c,d,e in get_similarity(question, 1):
        if c < 0.2:
          output = "Xin lỗi, không có dữ liệu về câu hỏi tương đồng mà tôi biết."
        else:
          output = f"Bạn có thể tham khảo câu trả lời sau:\n Câu hỏi: {a}\nTrả lời: {b}\nBạn có thể tham khảo đầy đủ tại đây: {d}\n\nHy vọng bạn sẽ hài lòng!"
      return output
    def respond(message, chat_history):
        bot_message = generate(message)
        chat_history.append((message, bot_message))
        time.sleep(2)
        return "", chat_history

    msg.submit(respond, [msg, chatbot], [msg, chatbot])

if __name__ == "__main__":
    demo.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://d40327b120446596ff.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [ ]:
import gradio as gr
import random
import time

with gr.Blocks() as demo:
    chatbot = gr.Chatbot()
    msg = gr.Textbox()
    clear = gr.ClearButton([msg, chatbot])

    def generate(question, current_index=0):
        similar_responses = get_similarity(question, 5)  # Change 5 to your desired number of similar responses

        if current_index < len(similar_responses):
            a, b, c, d, e = similar_responses[current_index]
            output = f"Bạn có thể tham khảo câu trả lời sau:\n Câu hỏi: {a}\nTrả lời: {b}\nTham khảo đầy đủ tại đây: {d}\n\nHy vọng bạn sẽ hài lòng!"
        else:
            output = "Không có thêm câu trả lời tương đồng."

        return output

    def respond(message, chat_history):
        # Assume you want to show the first set of similar responses initially
        bot_message = generate(message)
        chat_history.append((message, bot_message))
        time.sleep(2)
        return "", chat_history

    # Handle the "Xem thêm" button click
    def show_more(_, chat_history, current_index):
        # Increment the current_index to show the next set of similar responses
        current_index += 1
        bot_message = generate(chat_history[-1][0], current_index)
        chat_history[-1] = (chat_history[-1][0], bot_message)
        return "", chat_history, current_index

    msg.submit(respond, [msg, chatbot], [msg, chatbot])

    # Add a button to show more similar responses
    show_more_button = gr.Button("Xem thêm")
    show_more_button.click(show_more, [msg, chatbot], [msg, chatbot])

if __name__ == "__main__":
    demo.launch()


/usr/local/lib/python3.10/dist-packages/gradio/utils.py:805: UserWarning: Expected 3 arguments for function <function show_more at 0x7fe808a61fc0>, received 2.
  max_args += 1
/usr/local/lib/python3.10/dist-packages/gradio/utils.py:809: UserWarning: Expected at least 3 arguments for function <function show_more at 0x7fe808a61fc0>, received 2.
  return f"Keyword-only args must have default values for function {fn}"


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://e04476b8dca1b4dbab.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
